In [168]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv("dataset_eeg_nseg.csv")

In [4]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [5]:
df = df.set_index(['id_user','session','video'])

In [6]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      0.000056  0.000210  0.000578  0.001068  0.001386   
                2      0.000073  0.000045  0.000349  0.000667  0.000553   
                3      0.000053  0.000243  0.000697  0.000991  0.000822   
                4      0.000032  0.000114  0.000367  0.000646  0.000741   
                5      0.000037  0.000115  0.000549  0.000856  0.000560   
...                         ...       ...       ...       ...       ...   
9       3       20     0.000005  0.000028  0.000144  0.000136  0.000052   
                21     0.000012  0.000020  0.000093  0.000228  0.000360   
                22     0.000006  0.000026  0.000080  0.000192  0.000179   
                23     0.000035  0.000116  0.000271  0.000244  0.000255   
                24     0.000016  0.000144  0.000096  0.000285  0.000275   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      0.001648  0.000803  0.000437  0.000751  0.000383  ...   
                2      0.000305  0.000502  0.000579  0.000626  0.000521  ...   
                3      0.000288  0.000170  0.000315  0.000519  0.000392  ...   
                4      0.000520  0.000351  0.000349  0.000443  0.000537  ...   
                5      0.000430  0.000418  0.000234  0.000129  0.000227  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.000212  0.000210  0.000047  0.000127  0.000165  ...   
                21     0.000446  0.000264  0.000091  0.000087  0.000094  ...   
                22     0.000068  0.000094  0.000052  0.000129  0.000117  ...   
                23     0.000180  0.000203  0.000097  0.000168  0.000173  ...   
                24     0.000113  0.000217  0.000480  0.000640  0.000708  ...   

                           5502      5503      5504      5505      5506  \
id_user session video                                                     
10      1       1      0.680894  0.677372  0.678587  0.680154  0.679000   
                2      0.681449  0.680814  0.681055  0.679858  0.679219   
                3      0.680556  0.679891  0.680421  0.680982  0.680714   
                4      0.681528  0.681170  0.682441  0.681751  0.681922   
                5      0.681270  0.679830  0.681256  0.682198  0.681864   
...                         ...       ...       ...       ...       ...   
9       3       20     0.683626  0.684026  0.683992  0.683872  0.683737   
                21     0.683333  0.683626  0.683485  0.683547  0.683689   
                22     0.683917  0.683562  0.683447  0.683692  0.683361   
                23     0.683632  0.683680  0.683738  0.683844  0.683406   
                24     0.683428  0.683518  0.683481  0.683462  0.683615   

                           5507      5508      5509      5510  emotion  
id_user session video                                                   
10      1       1      0.679539  0.679375  0.678674  0.680591        1  
                2      0.680615  0.676147  0.680797  0.679779        2  
                3      0.680317  0.680449  0.680310  0.679916        3  
                4      0.682117  0.682258  0.681055  0.681194        0  
                5      0.680023  0.680439  0.680000  0.679143        2  
...                         ...       ...       ...       ...      ...  
9       3       20     0.683521  0.683858  0.683820  0.683911        0  
                21     0.683664  0.683373  0.683693  0.684037        2  
                22     0.683313  0.683518  0.684135  0.684174        0  
                23     0.683448  0.683663  0.683562  0.683567        1  
                24     0.684293  0.684325  0.684048  0.683871        0  

[1080 rows x 5512 columns]

## Split train e test
### Subject Dependent
16 trials train e 8 test
2 testing group for labels 

In [12]:
def split_sub_dep(df):    
    df_sd = df.reset_index()
    # generiamo 8 video su cui eseguiamo i test
    test_videos = np.random.randint(1, 25, size=8)
    print(test_videos)
    X_testSD = df_sd[df_sd['video'].isin(test_videos)]
    print(X_testSD.emotion.unique()) #check se sono rappresentate tutte le emozioni
    X_testSD.set_index(['id_user','session','video'], inplace=True)
    y_testSD = X_testSD.emotion
    X_testSD = X_testSD.drop(['emotion'],axis=1)
    X_trainSD = df_sd[~df_sd['video'].isin(test_videos)]
    X_trainSD.set_index(['id_user','session','video'], inplace=True)
    y_trainSD = X_trainSD.emotion
    X_trainSD = X_trainSD.drop(['emotion'],axis=1)
    return X_testSD,y_testSD,X_testSD,y_testSD

In [13]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[10 22  5 24 20 10 20 20]
[2 1 3 0]


In [8]:
clfSD = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0, max_depth=700,random_state=0).fit(X_trainSD, y_trainSD)

KeyboardInterrupt: 

In [ ]:
clfSD.score(X_testSD, y_testSD)

### Subject Indipendent
Leave One Out Cross Validation sugli utenti

In [53]:
def SubjectIndipendent_GBC(df):
    df_SI = df.reset_index()
    groups = df_SI['id_user']
    X_SI = df_SI.drop(['emotion','id_user','session','video'],axis=1)
    y_SI = df_SI.emotion
    return X_SI,y_SI,groups

In [54]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [ ]:
df_SI = df.reset_index()
groups = df_SI['id_user']

In [ ]:
groups

In [ ]:
X_SI = df_SI.drop(['emotion','id_user','session','video'],axis=1)

In [ ]:
X_SI

In [ ]:
y_SI = df_SI.emotion

In [ ]:
y_SI

#### GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.5, max_depth=100, random_state=0)

In [ ]:
logo = LeaveOneGroupOut()

In [ ]:
logo.get_n_splits(X_SI, y_SI, groups)

In [ ]:
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)

In [ ]:
scores

In [ ]:
scori = pd.DataFrame(scores)
scori.to_csv("scores.csv")

In [ ]:
scores.mean()

### Subject-Biased Experiment
train_test_split

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [ ]:
X_testSB

#### GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_trainSB, y_trainSB)

In [ ]:
clf.score(X_testSB, y_testSB)

# GRADIENT BOOSTING ON ONE FEATURE
### PSD

In [101]:
df = pd.read_csv("dataset_eeg_psds.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [31]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      0.000056  0.000210  0.000578  0.001068  0.001386   
                2      0.000073  0.000045  0.000349  0.000667  0.000553   
                3      0.000053  0.000243  0.000697  0.000991  0.000822   
                4      0.000032  0.000114  0.000367  0.000646  0.000741   
                5      0.000037  0.000115  0.000549  0.000856  0.000560   
...                         ...       ...       ...       ...       ...   
9       3       20     0.000005  0.000028  0.000144  0.000136  0.000052   
                21     0.000012  0.000020  0.000093  0.000228  0.000360   
                22     0.000006  0.000026  0.000080  0.000192  0.000179   
                23     0.000035  0.000116  0.000271  0.000244  0.000255   
                24     0.000016  0.000144  0.000096  0.000285  0.000275   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      0.001648  0.000803  0.000437  0.000751  0.000383  ...   
                2      0.000305  0.000502  0.000579  0.000626  0.000521  ...   
                3      0.000288  0.000170  0.000315  0.000519  0.000392  ...   
                4      0.000520  0.000351  0.000349  0.000443  0.000537  ...   
                5      0.000430  0.000418  0.000234  0.000129  0.000227  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.000212  0.000210  0.000047  0.000127  0.000165  ...   
                21     0.000446  0.000264  0.000091  0.000087  0.000094  ...   
                22     0.000068  0.000094  0.000052  0.000129  0.000117  ...   
                23     0.000180  0.000203  0.000097  0.000168  0.000173  ...   
                24     0.000113  0.000217  0.000480  0.000640  0.000708  ...   

                               2997          2998          2999          3000  \
id_user session video                                                           
10      1       1      3.907126e-16  3.465264e-16  7.531626e-16  8.150039e-16   
                2      6.520476e-16  1.179994e-15  1.722545e-15  9.845001e-16   
                3      2.156727e-15  2.189874e-15  2.041527e-15  2.554210e-15   
                4      2.606581e-16  2.907424e-16  4.283902e-16  2.684261e-16   
                5      7.830424e-16  3.084336e-16  6.882323e-17  1.386694e-16   
...                             ...           ...           ...           ...   
9       3       20     6.012194e-16  6.472331e-16  5.455448e-16  5.255017e-16   
                21     3.765455e-16  2.506991e-16  2.255052e-16  2.952234e-16   
                22     6.704449e-16  5.396330e-16  3.888893e-16  2.603666e-16   
                23     4.448681e-16  6.136698e-16  4.407513e-16  2.249349e-16   
                24     1.781403e-15  4.110112e-15  4.614508e-15  3.131334e-15   

                               3001          3002          3003          3004  \
id_user session video                                                           
10      1       1      7.625437e-16  8.066752e-16  2.697325e-16  4.403131e-16   
                2      4.387977e-16  7.747705e-16  3.279113e-16  8.037468e-16   
                3      2.231276e-15  2.258387e-15  2.284329e-15  2.668339e-15   
                4      2.439549e-16  4.774280e-16  4.943091e-16  2.360715e-16   
                5      4.272222e-16  3.145653e-16  1.443120e-16  3.482556e-16   
...                             ...           ...           ...           ...   
9       3       20     3.257976e-16  4.401326e-16  5.942757e-16  5.133194e-16   
                21     4.400796e-16  4.798846e-16  4.913319e-16  3.590445e-16   
                22     3.663340e-16  2.879779e-16  3.603275e-16  2.566137e-16   
                23     1.790649e-16  2.788588e-16  3.260241e-16  3.9

## DEPENDENCY

In [102]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[ 4  2 23  7 22 24  6 12]
[2 0 1 3]


In [156]:
clfSD = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=7,random_state=22).fit(X_trainSD, y_trainSD)

In [157]:
clfSD.score(X_trainSD, y_trainSD)

0.8833333333333333

In [158]:
clfSD.score(X_testSD, y_testSD)

0.8833333333333333

In [159]:
y_pred = clfSD.predict(X_testSD)

In [160]:
print(classification_report(y_testSD,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90        75
           1       0.81      0.99      0.89        90
           2       0.94      0.81      0.87        90
           3       0.86      0.90      0.88       105

    accuracy                           0.88       360
   macro avg       0.90      0.88      0.88       360
weighted avg       0.89      0.88      0.88       360



## INDIPENDENCY

In [107]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [108]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 12.0min finished


0.31851851851851853


## BIAS

In [109]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [110]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [111]:
clf.score(X_testSB, y_testSB)

0.3472222222222222

# GRADIENT BOOSTING ON ONE FEATURE
### DE

In [112]:
df = pd.read_csv("dataset_eeg_des.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [140]:
df

0             1             2             3  \
id_user session video                                                           
10      1       1      12650.603507  12648.390584  12627.183801  12644.366178   
                2      23159.311016  23228.675318  23200.891697  23214.945153   
                3      20310.108415  20344.375682  20342.634717  20324.171070   
                4      21714.090429  21693.944995  21692.633774  21691.334161   
                5      26387.909596  26468.273255  26470.628686  26474.199258   
...                             ...           ...           ...           ...   
9       3       20     83232.652198  83262.416482  83271.476343  83269.486399   
                21     44377.498193  44368.071520  44360.024699  44375.239472   
                22     77909.122398  77912.585034  77901.294744  77902.037879   
                23     57872.923314  57867.404649  57869.875955  57873.861769   
                24     23737.507729  23703.337772  23683.724240  23719.810333   

                                  4             5             6             7  \
id_user session video                                                           
10      1       1      12655.821677  12657.912527  12657.423942  12670.734588   
                2      23185.253969  23197.676893  23220.022194  23230.254935   
                3      20358.065116  20380.788349  20358.704419  20326.576719   
                4      21688.980463  21705.762150  21716.607324  21682.026333   
                5      26474.122085  26464.252362  26501.362420  26497.658895   
...                             ...           ...           ...           ...   
9       3       20     83264.655727  83254.617406  83263.672846  83221.614911   
                21     44372.010338  44366.725660  44361.905632  44366.783955   
                22     77908.918529  77915.819825  77903.441838  77892.644420   
                23     57874.105052  57877.119209  57867.815270  57866.116360   
                24     23718.304043  23724.796173  23744.506479  23739.200124   

                                  8             9  ...           1494  \
id_user session video                              ...                  
10      1       1      12671.490607  12675.944154  ...   18690.330810   
                2      23243.865618  23243.604063  ...   34293.059359   
                3      20340.163577  20371.347796  ...   30060.106190   
                4      21727.137159  21729.282150  ...   32035.791191   
                5      26528.326749  26475.762155  ...   39129.674113   
...                             ...           ...  ...            ...   
9       3       20     83229.445640  83261.786274  ...  122478.488712   
                21     44369.859135  44363.422629  ...   65272.166365   
                22     77902.196130  77897.672069  ...  114614.792502   
                23     57870.890395  57872.707613  ...   85137.511743   
                24     23733.973069  23731.417918  ...   34909.847325   

                                1495           1496           1497  \
id_user session video                                                
10      1       1       18690.888346   18690.699299   18690.448836   
                2       34293.244473   34293.176953   34293.509391   
                3       30060.274793   30060.127199   30059.981530   
                4       32035.900896   32035.502696   32035.721090   
                5       39130.153228   39129.623449   39129.347652   
...                              ...            ...            ...   
9       3       20     122478.000338  122478.037504  122478.177637   
                21      65271.981952   65272.082634   65272.035110   
                22     114615.197337  114615.328005  114615.032510   
                23      85137.471387   85137.422563   85137.324035   
                24      34909.817159   34909.830729   34909.836524   

                                1498           1499           150

## DEPENDENCY

In [113]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[13 11  9  8  5 11 24 14]
[2 1 0 3]


In [162]:
clfSD = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=7,random_state=10).fit(X_trainSD, y_trainSD)

In [163]:
clfSD.score(X_testSD, y_testSD)

0.8777777777777778

In [164]:
y_pred = clfSD.predict(X_testSD)

In [165]:
print(classification_report(y_testSD,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90        75
           1       0.81      0.99      0.89        90
           2       0.95      0.81      0.87        90
           3       0.84      0.88      0.86       105

    accuracy                           0.88       360
   macro avg       0.89      0.88      0.88       360
weighted avg       0.89      0.88      0.88       360



## INDIPENDENCY

In [72]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [73]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0.3157407407407407


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.1min finished


## BIAS

In [118]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [119]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [120]:
clf.score(X_testSB, y_testSB)

0.26851851851851855

# GRADIENT BOOSTING ON ONE FEATURE
### ASM

In [121]:
df = pd.read_csv("dataset_eeg_asms.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [122]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1     -0.507886 -0.614872 -1.641967 -0.809440 -0.254521   
                2     -1.526726 -0.019622 -0.623337 -0.318292 -0.963419   
                3     -1.712901 -0.626136 -0.681488 -1.267282 -0.191965   
                4     -0.326937 -1.115830 -1.166505 -1.217275 -1.309265   
                5     -2.620650 -0.919027 -0.869191 -0.793411 -0.794974   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.363106  0.264170  0.455100  0.413153  0.311342   
                21     0.845171  0.200099 -0.350465  0.690646  0.469665   
                22     0.101389  0.227249 -0.183157 -0.156111  0.094033   
                23     0.296960  0.046466  0.158650  0.339566  0.350600   
                24     0.984481 -2.887236 -5.109027 -1.020892 -1.191568   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1     -0.153257 -0.177033  0.468183  0.504833  0.720677  ...   
                2     -0.693255 -0.207848  0.014371  0.310071  0.304314  ...   
                3      0.529379 -0.171602 -1.190944 -0.759927  0.229540  ...   
                4     -0.652792 -0.228270 -1.581202  0.183983  0.267916  ...   
                5     -1.004569 -0.218292 -0.296775  0.353089 -0.760424  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     0.099790  0.290642 -0.595710 -0.430711  0.250864  ...   
                21     0.108025 -0.221799  0.112040  0.322455 -0.118011  ...   
                22     0.344920 -0.105058 -0.497562 -0.150350 -0.314806  ...   
                23     0.487405  0.065113 -0.011989  0.204698  0.287173  ...   
                24    -0.455939  1.777643  1.176339  0.583990  0.294424  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -0.603754 -3.936397 -2.786970 -1.303838 -2.395775   
                2     -0.041316 -0.535183 -0.348228 -1.279361 -1.776960   
                3     -1.006256 -1.668214 -1.141022 -0.581752 -0.849187   
                4     -0.820992 -1.289149  0.374519 -0.529157 -0.304496   
                5     -0.323980 -1.570919 -0.336004  0.479314  0.190322   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.236637  0.839615  0.747818  0.426037  0.061812   
                21    -1.719179 -0.353402 -1.006594 -0.720096 -0.058573   
                22     0.837657 -0.709266 -1.209290 -0.144284 -1.580553   
                23    -0.221597 -0.028114  0.207154  0.636081 -1.132681   
                24    -1.218383 -0.846308 -0.999152 -1.075692 -0.446103   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -1.886432 -2.041642 -2.704514 -0.891223        1  
                2     -0.690190 -4.166999 -0.548480 -1.340570        2  
                3     -1.244093 -1.113199 -1.251479 -1.643447        3  
                4     -0.049593  0.135184 -1.440402 -1.258214        0  
                5     -1.403735 -1.043561 -1.423911 -2.165414        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.519885  0.388118  0.286195  0.531003        0  
                21    -0.172256 -1.529113 -0.037419  1.565252        2  
                22    -1.788901 -0.900419  1.783097  1.951517        0  
                23    -0.960791 -0.096051 -0.502811 -0.483703        1  
                24     2.355415  2.488038  1.343486  0.613903        0  

[1080 rows x 502 columns]

## DEPENDENCY

In [123]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[ 7  4  7 19 11  6 23 10]
[0 1 2 3]


In [124]:
clfSD = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=2,random_state=0).fit(X_trainSD, y_trainSD)

In [125]:
clfSD.score(X_testSD, y_testSD)

0.6666666666666666

In [126]:
y_pred = clfSD.predict(X_testSD)

In [127]:
print(classification_report(y_testSD,y_pred))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       120
           1       1.00      0.43      0.60        60
           2       1.00      0.38      0.55        60
           3       0.98      0.55      0.70        75

    accuracy                           0.67       315
   macro avg       0.88      0.59      0.64       315
weighted avg       0.82      0.67      0.65       315



## INDIPENDENCY

In [84]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [85]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


0.2685185185185185


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.9min finished


## BIAS

In [128]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [129]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [130]:
clf.score(X_testSB, y_testSB)

0.3101851851851852

# GRADIENT BOOSTING ON ONE FEATURE
### DASM

In [131]:
df = pd.read_csv("dataset_eeg_dasms.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [141]:
df

0             1             2             3  \
id_user session video                                                           
10      1       1      12650.603507  12648.390584  12627.183801  12644.366178   
                2      23159.311016  23228.675318  23200.891697  23214.945153   
                3      20310.108415  20344.375682  20342.634717  20324.171070   
                4      21714.090429  21693.944995  21692.633774  21691.334161   
                5      26387.909596  26468.273255  26470.628686  26474.199258   
...                             ...           ...           ...           ...   
9       3       20     83232.652198  83262.416482  83271.476343  83269.486399   
                21     44377.498193  44368.071520  44360.024699  44375.239472   
                22     77909.122398  77912.585034  77901.294744  77902.037879   
                23     57872.923314  57867.404649  57869.875955  57873.861769   
                24     23737.507729  23703.337772  23683.724240  23719.810333   

                                  4             5             6             7  \
id_user session video                                                           
10      1       1      12655.821677  12657.912527  12657.423942  12670.734588   
                2      23185.253969  23197.676893  23220.022194  23230.254935   
                3      20358.065116  20380.788349  20358.704419  20326.576719   
                4      21688.980463  21705.762150  21716.607324  21682.026333   
                5      26474.122085  26464.252362  26501.362420  26497.658895   
...                             ...           ...           ...           ...   
9       3       20     83264.655727  83254.617406  83263.672846  83221.614911   
                21     44372.010338  44366.725660  44361.905632  44366.783955   
                22     77908.918529  77915.819825  77903.441838  77892.644420   
                23     57874.105052  57877.119209  57867.815270  57866.116360   
                24     23718.304043  23724.796173  23744.506479  23739.200124   

                                  8             9  ...           1494  \
id_user session video                              ...                  
10      1       1      12671.490607  12675.944154  ...   18690.330810   
                2      23243.865618  23243.604063  ...   34293.059359   
                3      20340.163577  20371.347796  ...   30060.106190   
                4      21727.137159  21729.282150  ...   32035.791191   
                5      26528.326749  26475.762155  ...   39129.674113   
...                             ...           ...  ...            ...   
9       3       20     83229.445640  83261.786274  ...  122478.488712   
                21     44369.859135  44363.422629  ...   65272.166365   
                22     77902.196130  77897.672069  ...  114614.792502   
                23     57870.890395  57872.707613  ...   85137.511743   
                24     23733.973069  23731.417918  ...   34909.847325   

                                1495           1496           1497  \
id_user session video                                                
10      1       1       18690.888346   18690.699299   18690.448836   
                2       34293.244473   34293.176953   34293.509391   
                3       30060.274793   30060.127199   30059.981530   
                4       32035.900896   32035.502696   32035.721090   
                5       39130.153228   39129.623449   39129.347652   
...                              ...            ...            ...   
9       3       20     122478.000338  122478.037504  122478.177637   
                21      65271.981952   65272.082634   65272.035110   
                22     114615.197337  114615.328005  114615.032510   
                23      85137.471387   85137.422563   85137.324035   
                24      34909.817159   34909.830729   34909.836524   

                                1498           1499           150

## DEPENDENCY

In [132]:
X_trainSD,y_trainSD,X_testSD,y_testSD = split_sub_dep(df)

[ 5  3  9 13 19  1 10 16]
[1 3 2 0]


In [133]:
clfSD = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=2,random_state=0).fit(X_trainSD, y_trainSD)

In [134]:
clfSD.score(X_testSD, y_testSD)

0.4111111111111111

In [135]:
y_pred = clfSD.predict(X_testSD)

In [136]:
print(classification_report(y_testSD,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.12      0.21        75
           1       0.62      0.31      0.41        90
           2       0.48      0.16      0.24        90
           3       0.35      0.92      0.51       105

    accuracy                           0.41       360
   macro avg       0.59      0.38      0.34       360
weighted avg       0.57      0.41      0.36       360



## INDIPENDENCY

In [96]:
X_SI,y_SI,groups = SubjectIndipendent_GBC(df)

In [97]:
model = GradientBoostingClassifier(n_estimators=20, learning_rate=1, max_depth=15, random_state=0)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_SI, y_SI, groups)
scores = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  5.0min finished


0.2814814814814815


## BIAS

In [137]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [138]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0).fit(X_trainSB, y_trainSB)

In [139]:
clf.score(X_testSB, y_testSB)

0.25925925925925924